# Hotel Booking Cancellation Analysis (Individual Component)

This notebook documents my individual analytical contribution to the final project.
The focus is on understanding drivers of hotel booking cancellations in Portugal
and translating those findings into model-driven insights that inform the final dashboard.

This notebook includes:
- Prompt logs and DIVE reflections for one substantive question
- Curated data creation
- A BigQuery ML cancellation model
- Model evaluation and explanation
- An interactive Plotly visualization
- A link to the dashboard section influenced by this analysis


## Substantive Question

Which booking characteristics most strongly influence whether a hotel reservation
in Portugal is canceled, and how can these insights be used to identify high-risk
future bookings?


In [2]:
!pip install google-cloud-bigquery pandas-gbq -q

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.cloud import bigquery

project_id = "mgmt-467-471819"
client = bigquery.Client(project=project_id)

query = """
SELECT *
FROM `mgmt-467-471819.hotel_batch.hotel_bookings_raw`
"""

df = client.query(query).to_dataframe()

df.head(), df.shape


(          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
 0  Resort Hotel            0        342               2015               July   
 1  Resort Hotel            0        737               2015               July   
 2  Resort Hotel            0          7               2015               July   
 3  Resort Hotel            0         13               2015               July   
 4  Resort Hotel            0         14               2015               July   
 
    arrival_date_week_number  arrival_date_day_of_month  \
 0                        27                          1   
 1                        27                          1   
 2                        27                          1   
 3                        27                          1   
 4                        27                          1   
 
    stays_in_weekend_nights  stays_in_week_nights  adults  ...  deposit_type  \
 0                        0                     0       2  ...    No D

In [5]:
import pandas as pd

# df is your raw table from BigQuery
# df = client.query("SELECT * FROM `mgmt-467-35946.hotel_booking.hotel_bookings`").to_dataframe()

# 0. Filter to only Portugal bookings
df = df[df["country"] == "PRT"].copy()

# 1. Map month names → month numbers
month_map = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}
df["arrival_month_num"] = df["arrival_date_month"].map(month_map)

# 2. Build a proper arrival_date column
df["arrival_date"] = pd.to_datetime({
    "year": df["arrival_date_year"],
    "month": df["arrival_month_num"],
    "day": df["arrival_date_day_of_month"]
})

# 3. Parse reservation_status_date
df["reservation_status_date"] = pd.to_datetime(df["reservation_status_date"])

# 4. Derived feature: total number of guests
df["children"] = df["children"].fillna(0)
df["total_guests"] = df["adults"] + df["children"] + df["babies"]

# 5. Data-quality filters (use adr before we drop it)
df_curated = df[(df["total_guests"] > 0) & (df["adr"] >= 0)].copy()

# 6. Drop helper and unwanted columns
cols_to_drop = ["arrival_month_num", "company", "agent", "adr", "meal"]
df_curated.drop(columns=cols_to_drop, inplace=True, errors="ignore")

# Take a look
df_curated.head(), df_curated.shape

(          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
 0  Resort Hotel            0        342               2015               July   
 1  Resort Hotel            0        737               2015               July   
 6  Resort Hotel            0          0               2015               July   
 7  Resort Hotel            0          9               2015               July   
 8  Resort Hotel            1         85               2015               July   
 
    arrival_date_week_number  arrival_date_day_of_month  \
 0                        27                          1   
 1                        27                          1   
 6                        27                          1   
 7                        27                          1   
 8                        27                          1   
 
    stays_in_weekend_nights  stays_in_week_nights  adults  ...  \
 0                        0                     0       2  ...   
 1                

In [6]:
import pandas_gbq

dataset_id = "hotel_batch"
table_id = "hotel_bookings_curated"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

pandas_gbq.to_gbq(
    df_curated,
    full_table_id,
    project_id=project_id,
    if_exists="replace"  # or "fail" if you don't want to overwrite
)

100%|██████████| 1/1 [00:00<00:00, 7244.05it/s]


In [7]:
query = """
SELECT COUNT(*) AS row_count
FROM `mgmt-467-471819.hotel_batch.hotel_bookings_curated`
"""

client.query(query).to_dataframe()

,row_count
0,48479


In [8]:
create_model_query = """
CREATE OR REPLACE MODEL `mgmt-467-471819.hotel_batch.cancellation_model`
OPTIONS (
  model_type = 'logistic_reg',
  input_label_cols = ['is_canceled'],
  data_split_method = 'RANDOM',
  data_split_eval_fraction = 0.2
) AS
SELECT
  is_canceled,
  hotel,
  lead_time,
  stays_in_weekend_nights,
  stays_in_week_nights,
  adults,
  children,
  babies,
  total_guests,
  is_repeated_guest,
  previous_cancellations,
  previous_bookings_not_canceled,
  booking_changes,
  days_in_waiting_list,
  required_car_parking_spaces,
  total_of_special_requests,
  market_segment,
  distribution_channel,
  customer_type,
  deposit_type,
  temperature_2m_mean,
  temperature_2m_max,
  temperature_2m_min,
  precipitation_sum,
  windspeed_10m_max,
  weathercode
FROM `mgmt-467-471819.hotel_batch.hotel_bookings_historical_weather`
WHERE country = 'PRT'
"""
client.query(create_model_query).result()

In [9]:
eval_query = """
SELECT *
FROM ML.EVALUATE(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`
)
"""
client.query(eval_query).to_dataframe()

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.832571,0.830444,0.810678,0.831506,0.376531,0.906621


In [10]:
weight_query = """
SELECT *
FROM ML.WEIGHTS(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`
)
WHERE processed_input NOT IN ('__INTERCEPT__')
"""
client.query(weight_query).to_dataframe()

,processed_input,weight,category_weights
0,hotel,NaN,"[{'category': 'City Hotel', 'weight': 0.173933..."
1,lead_time,0.004853,[]
2,stays_in_weekend_nights,0.186317,[]
3,stays_in_week_nights,0.155988,[]
4,adults,0.159735,[]
5,children,0.014444,[]
6,babies,-0.173391,[]
7,total_guests,0.119455,[]
8,is_repeated_guest,-0.729591,[]
9,previous_cancellations,0.245439,[]


In [11]:
predict_query = """
SELECT
  hotel,
  lead_time,
  total_guests,
  market_segment,
  deposit_type,
  predicted_is_canceled_probs[OFFSET(1)] AS cancel_probability
FROM ML.PREDICT(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`,
  (
    SELECT
      b.hotel,
      b.lead_time,
      b.stays_in_weekend_nights,
      b.stays_in_week_nights,
      b.adults,
      b.children,
      b.babies,
      b.total_guests,
      b.is_repeated_guest,
      b.previous_cancellations,
      b.previous_bookings_not_canceled,
      b.booking_changes,
      b.days_in_waiting_list,
      b.required_car_parking_spaces,
      b.total_of_special_requests,
      b.market_segment,
      b.distribution_channel,
      b.customer_type,
      b.deposit_type,
      w.temperature AS temperature_2m_mean,
      w.temperature AS temperature_2m_max,
      w.temperature AS temperature_2m_min,
      w.precipitation AS precipitation_sum,
      w.wind_speed AS windspeed_10m_max,
      0 AS weathercode
    FROM `mgmt-467-471819.hotel_batch.hotel_bookings_historical_weather` b
    CROSS JOIN (
      SELECT
        temperature,
        precipitation,
        wind_speed
      FROM `mgmt-467-471819.hotel_stream.weather_live`
      WHERE city = 'Lisbon'
      ORDER BY timestamp DESC
      LIMIT 1
    ) w
    WHERE b.country = 'PRT'
    LIMIT 50
  )
)
"""
client.query(predict_query).to_dataframe()

,hotel,lead_time,total_guests,market_segment,deposit_type,cancel_probability
0,City Hotel,11,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.14861781175849065}"
1,City Hotel,1,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.48535734247201723}"
2,City Hotel,3,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.0828853782205975}"
3,City Hotel,11,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.14861781175849065}"
4,City Hotel,5,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.4343029748584488}"
5,City Hotel,10,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.4744564999583052}"
6,City Hotel,3,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.16889151854652895}"
7,City Hotel,3,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.5510510932018267}"
8,City Hotel,5,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.5285122062190742}"
9,City Hotel,2,1,Aviation,No Deposit,"{'label': 0, 'prob': 0.4841452591260166}"


In [12]:
query = """SELECT *
FROM ML.WEIGHTS(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`
)
LIMIT 5;
"""
client.query(query).to_dataframe()

,processed_input,weight,category_weights
0,hotel,NaN,"[{'category': 'City Hotel', 'weight': 0.173933..."
1,lead_time,0.004853,[]
2,stays_in_weekend_nights,0.186317,[]
3,stays_in_week_nights,0.155988,[]
4,adults,0.159735,[]


In [13]:
explain_query = """
SELECT
  *
FROM ML.EXPLAIN_PREDICT(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`,
  (
    SELECT *
    FROM `mgmt-467-471819.hotel_batch.hotel_bookings_historical_weather`
    WHERE country = 'PRT'
    LIMIT 20
  ),
  STRUCT(
    0.5 AS threshold
  )
)
"""
explain_df = client.query(explain_query).to_dataframe()
explain_df

,predicted_is_canceled,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,hotel,is_canceled,lead_time,arrival_date_year,...,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,windspeed_10m_max,weathercode,weather_source,weather_loaded_at
0,0,0.859924,"[{'feature': 'required_car_parking_spaces', 'a...",0.120039,-1.814661,0.0,City Hotel,0,0,2016,...,38.72,-9.14,12.5,9.4,11.0,3.0,48.5,53,open-meteo-archive,2025-12-12 22:13:02.651967+00:00
1,0,0.941708,"[{'feature': 'required_car_parking_spaces', 'a...",0.120039,-2.782234,0.0,City Hotel,0,2,2016,...,38.72,-9.14,15.0,10.7,12.5,0.0,16.6,3,open-meteo-archive,2025-12-12 22:13:12.075296+00:00
2,0,0.892440,"[{'feature': 'required_car_parking_spaces', 'a...",0.120039,-2.115908,0.0,City Hotel,0,7,2016,...,38.72,-9.14,14.9,8.5,11.3,2.6,22.0,61,open-meteo-archive,2025-12-12 22:13:12.075296+00:00
3,0,0.561835,"[{'feature': 'lead_time', 'attribution': -0.53...",0.120039,-0.248613,0.0,City Hotel,1,5,2016,...,38.72,-9.14,14.0,11.9,13.2,8.2,37.5,61,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
4,0,0.985287,"[{'feature': 'required_car_parking_spaces', 'a...",0.120039,-4.204187,0.0,City Hotel,0,5,2016,...,38.72,-9.14,14.9,11.4,12.9,0.0,34.9,3,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
5,0,0.630753,"[{'feature': 'lead_time', 'attribution': -0.55...",0.120039,-0.535448,0.0,City Hotel,0,1,2016,...,38.72,-9.14,17.9,10.8,13.7,0.0,27.3,2,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
6,0,0.507132,"[{'feature': 'lead_time', 'attribution': -0.53...",0.120039,-0.028530,0.0,City Hotel,0,6,2016,...,38.72,-9.14,18.5,11.9,14.3,0.0,38.4,3,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
7,0,0.511983,"[{'feature': 'lead_time', 'attribution': -0.55...",0.120039,-0.047941,0.0,City Hotel,0,2,2016,...,38.72,-9.14,18.5,11.9,14.3,0.0,38.4,3,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
8,1,0.617256,"[{'feature': 'lead_time', 'attribution': -0.53...",0.120039,0.477918,0.0,City Hotel,0,5,2016,...,38.72,-9.14,13.5,9.7,12.0,1.3,33.1,51,open-meteo-archive,2025-12-12 22:13:16.211819+00:00
9,0,0.934348,"[{'feature': 'required_car_parking_spaces', 'a...",0.120039,-2.655483,0.0,City Hotel,0,11,2016,...,38.72,-9.14,13.5,9.7,12.0,1.3,33.1,51,open-meteo-archive,2025-12-12 22:13:16.211819+00:00


In [14]:
explain_df["top_feature_attributions"].iloc[0]

array([{'feature': 'required_car_parking_spaces', 'attribution': -2.1633007111687474},
       {'feature': 'lead_time', 'attribution': -0.5624089001623909},
       {'feature': 'market_segment', 'attribution': 0.5235224494470565},
       {'feature': 'deposit_type', 'attribution': -0.5102540559722479},
       {'feature': 'customer_type', 'attribution': 0.27531147450830695}],
      dtype=object)

In [15]:
weather_proof = """
SELECT
  processed_input,
  weight
FROM ML.WEIGHTS(
  MODEL `mgmt-467-471819.hotel_batch.cancellation_model`
)
WHERE processed_input LIKE '%temp%'
   OR processed_input LIKE '%precip%'
   OR processed_input LIKE '%wind%'
   OR processed_input LIKE '%weather%'
ORDER BY ABS(weight) DESC;
"""
client.query(weather_proof).to_dataframe()

,processed_input,weight
0,temperature_2m_min,-0.004840
1,temperature_2m_max,0.004522
2,windspeed_10m_max,-0.004411
3,precipitation_sum,-0.001173
4,temperature_2m_mean,-0.000491
5,weathercode,0.000041


## Interactive Weather Time Series (Streaming Data)

This interactive Plotly visualization shows live weather conditions in Lisbon over time.
It is used to validate the streaming weather inputs that feed into the cancellation
prediction model and to ensure values are within reasonable ranges.


In [17]:
weather_ts_query = """
SELECT
  timestamp,
  temperature AS temperature_c,
  wind_speed AS wind_speed_ms
FROM `mgmt-467-471819.hotel_stream.weather_live`
WHERE city = 'Lisbon'
ORDER BY timestamp
LIMIT 500
"""

weather_ts_df = client.query(weather_ts_query).to_dataframe()
weather_ts_df["timestamp"] = pd.to_datetime(weather_ts_df["timestamp"])

weather_ts_df.tail()


,timestamp,temperature_c,wind_speed_ms
300,2025-12-14 23:00:04.998418+00:00,13.1,6.4
301,2025-12-14 23:10:04.367016+00:00,13.1,6.4
302,2025-12-14 23:20:04.353982+00:00,12.9,6.1
303,2025-12-14 23:30:04.445482+00:00,12.9,6.1
304,2025-12-14 23:40:04.286754+00:00,12.9,6.1


In [18]:
import plotly.graph_objects as go

fig = go.Figure()

# Temperature line (left axis)
fig.add_trace(
    go.Scatter(
        x=weather_ts_df["timestamp"],
        y=weather_ts_df["temperature_c"],
        name="Temperature (Degrees Celsius)",
        mode="lines",
        line=dict(width=3)
    )
)

# Wind speed line (right axis)
fig.add_trace(
    go.Scatter(
        x=weather_ts_df["timestamp"],
        y=weather_ts_df["wind_speed_ms"],
        name="Wind Speed (m/s)",
        mode="lines",
        line=dict(width=3),
        yaxis="y2"
    )
)

fig.update_layout(
    title="Weather Time Series",
    xaxis=dict(title="Timestamp"),
    yaxis=dict(
        title="Temperature (Degrees Celsius)",
        side="left"
    ),
    yaxis2=dict(
        title="Wind Speed (m/s)",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig.show()


Looker Studio Link: https://lookerstudio.google.com/u/0/reporting/4ae22beb-3898-47f3-b5f0-7b6097c70f82/page/yKjiF/edit

This chart was used to confirm that live temperature and wind speed values used in
real-time predictions are stable and consistent with historical ranges observed
in the training data.


## DIVE Reflection

### Discover
The initial discovery from the cancellation model was that booking behavior variables dominate cancellation risk, not external conditions. The strongest drivers were operational features such as required_car_parking_spaces, booking_changes, is_repeated_guest, and total_of_special_requests. In contrast, weather-related variables including temperature, precipitation, and wind speed had coefficients close to zero, indicating minimal direct influence on cancellation outcomes.

### Investigate
To investigate whether weather effects were being understated due to data issues, I examined both historical weather features and live streaming weather inputs. Alternative queries isolating weather-related model weights confirmed that all weather variables had negligible magnitudes relative to booking characteristics. This shifted the analytical focus away from environmental drivers and toward customer intent and booking flexibility.

### Validate
AI-generated SQL could have been incorrect or misleading if weather data was improperly joined, scaled inconsistently, or outside realistic ranges. To validate this, I used an interactive Plotly time-series chart built from live streaming weather data for Lisbon. This visualization confirmed that real-time temperature and wind speed values were stable and aligned with historical distributions used during model training. Additionally, direct inspection of model weights filtered to weather variables verified that their low impact was genuine rather than a data artifact.

### Extend
These findings can be used immediately by hotel revenue management and operations teams. Rather than reacting to weather conditions, stakeholders should focus on booking-level risk signals, such as customers with no deposits, frequent booking changes, or low commitment indicators. The live weather chart can remain part of the dashboard as a validation and monitoring tool, while the primary action should be to flag high-risk reservations for proactive interventions like deposits or confirmation reminders.


## Prompt Log (AI-Assisted Analysis)

- You are a data analyst working with hotel booking data in BigQuery. Help me write SQL to extract raw hotel booking records and load them into a pandas DataFrame in Google Colab.

- You are a data analyst. Given hotel booking data with separate year, month name, and day fields, help me construct a proper arrival_date column in pandas.

- You are a data analyst reviewing data quality. Identify appropriate filters to remove invalid hotel bookings, such as records with zero guests or negative pricing.

- You are a data analyst using BigQuery ML. Help me create a logistic regression model to predict booking cancellations, including selecting relevant booking and weather features.

- You are a data analyst interpreting a logistic regression model. Explain how to retrieve and interpret model weights from a BigQuery ML cancellation model.

- You are a data analyst validating model inputs. Help me check whether weather variables meaningfully contribute to cancellation predictions or if their impact is negligible.

- You are a data analyst working with streaming data. Help me query live weather data from a BigQuery streaming table and load it into pandas for visualization.

- You are a data analyst creating interactive visualizations. Help me build a Plotly time-series chart with dual y-axes to visualize live temperature and wind speed data.

- You are a data analyst connecting analysis to business decisions. Help me translate cancellation model results into actionable insights for hotel revenue management.
